### Problem
1. It contains 4 columns and 3 rows
2. Position 1,4 has Queen
3. Position 2,4 has firepit
4. Position 2,2 has wall

In [1]:
import numpy as np

In [2]:
"""
Our Grid Object
"""
class Grid:
    def __init__(self,width,height,start):
        self.width = width
        self.height = height
        self.i = start[0] #vertical axis
        self.j = start[1] #horizontal axis
    
    def set(self,actions,rewards,obey_prob):
        self.actions = actions # dict:  (row,col): list of actions
        self.rewards = rewards # dict:  (row,col):  reward
        self.obey_prob = obey_prob
    
    def non_terminal_states(self):
        return self.actions.keys()
    
    def set_state(self,s):
        #change current stae
        self.i = s[0]
        self.j = s[1]
    
    def current_state(self):
        return (self.i,self.j)
    
    def is_terminal(self,s):
        return s not in self.actions
    
    def check_move(self,action):
        i = self.i
        j = self.j
        #check move validity
        if action in self.actions[(i,j)]:
            if action == "U":
                i-=1
            elif action == "D":
                i+=1
            elif action == "R":
                j+=1
            elif action == "L":
                j-=1
                
        reward = self.rewards.get((i,j),0)
        return ((i,j),reward)
    
    def get_transition_probs(self,action):
        # returns a list of (probability, reward, s') transition tuples
        probs = []
        state, reward = self.check_move(action)
        probs.append((self.obey_prob,reward,state))
        
        """
        if obey prob is 0.8
        and if action is 'up' then 
        'up' has probabiltiy of 0.8
        'left' has probability of 0.1
        'right' has probability of 0.1
        """
        disobey_prob = 1-self.obey_prob
        if disobey_prob <= 0:
            return probs
        if action == "U" or action == "D":
            state, reward = self.check_move("L")
            probs.append((disobey_prob/2,reward,state))
            state, reward = self.check_move("R")
            probs.append((disobey_prob/2,reward,state))
        elif action == "L" or action == "R":
            state, reward = self.check_move("U")
            probs.append((disobey_prob/2,reward,state))
            state,reward = self.check_move("D")
            probs.append((disobey_prob/2,reward,state))
        return probs
    
    def game_over(self):
        #return True if game is over else False
        #true if we are in state where no action are possible
        return (self.i,self.j) not in self.actions.keys()
    
    def all_states(self):
        # get all states
        # either a position that has possible next actions
        # or a position that yields a reward
        return set(self.actions.keys()) | set(self.rewards.keys())

In [3]:
"""
Initialization of Grid
"""
def standard_grid(obey_prob=0.1, step_cost = None):
    # lets define a grid that describes the reward for getting into each state
    # and possible actions at each state
    # the grid looks like this
    # x means you can't go there
    # s means start position
    # number means reward at that state
    # .  .  .  1
    # .  x  . -1
    # s  .  .  .
    # obey_brob (float): the probability of obeying the command
    # step_cost (float): a penalty applied each step to minimize the number of moves (-0.1)
    g = Grid(3, 4, (2, 0))
    
    rewards = {(0,3):+1, (1,3):-1}
    start = (2,0)
    actions = {
        (0,0) : ["R","D"],
        (0,1) : ["L","R"],
        (0,2) : ["L","D","R"],
        (1,0) : ["U","D"],
        (1,2) : ["U","D","R"],
        (2,0) : ["U","R"],
        (2,1) : ["L","R"],
        (2,2) : ["L","U","R"],
        (2,3) : ["L","U"]
    }
    
    g.set(actions,rewards,obey_prob)
    if step_cost is not None:
        g.rewards.update({
          (0, 0): step_cost,
          (0, 1): step_cost,
          (0, 2): step_cost,
          (1, 0): step_cost,
          (1, 2): step_cost,
          (2, 0): step_cost,
          (2, 1): step_cost,
          (2, 2): step_cost,
          (2, 3): step_cost,
        })
    return g

In [4]:
"""
Displaying Results
"""
def print_values(V, g):
    for i in range(g.width):
        print("---------------------------")
        for j in range(g.height):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")

def print_policy(P, g):
    for i in range(g.width):
        print("---------------------------")
        for j in range(g.height):
            a = P.get((i,j), ' ')
            print("  %s  |" % a, end="")
        print("")

In [5]:
"""
Value-Iteration
"""
SMALL_ENOUGH = 1e-3
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ["U","D","R","L"]

def best_action_value(grid,V,s):
    # finds the highest value action (max_a) from state s, returns the action and value
    best_a = None
    best_value = float('-inf')
    grid.set_state(s)
    
    #loop through all possible action to find the best possible actions
    for action in ALL_POSSIBLE_ACTIONS:
        transitions = grid.get_transition_probs(action)
        expected_v = 0
        expected_r = 0
        for (prob,r,state_prime) in transitions:
            expected_v += prob*V[state_prime]
            expected_r += prob*r
        v = expected_r + GAMMA* expected_v
        if v > best_value:
            best_value = v
            best_a = action
    return (best_a,best_value)


def calculate_values(grid):
    V = {}
    states = grid.all_states()
    for s in states:
        V[s] = 0
    
    #repeat until convergence
    #V[s] = max[a]{ sum[s',r] { p(s',r|s,a)[r + gamma*V[s']] } }
    while True:
        biggest_change = 0
        for s in grid.non_terminal_states():
            old_v = V[s]
            _,new_v = best_action_value(grid,V,s)
            V[s] = new_v
            biggest_change = max(biggest_change,np.abs(old_v-new_v))
        if biggest_change <= SMALL_ENOUGH:
            break;
    return V

In [6]:
def initialize_random_policy():
    # policy is a lookup table for state -> action
    # we'll randomly choose an action and update as we learn
    policy = {}
    for s in grid.non_terminal_states():
        policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
    return policy


def calculate_greedy_policy(grid, V):
    policy = initialize_random_policy()
    # find a policy that leads to optimal value function
    for s in policy.keys():
        grid.set_state(s)
        # loop through all possible actions to find the best current action
        best_a, _ = best_action_value(grid, V, s)
        policy[s] = best_a
    return policy

In [7]:
def print_game(grid):
    # print rewards
    print("rewards:")
    print_values(grid.rewards, grid)

    # calculate accurate values for each square
    V = calculate_values(grid)

    # calculate the optimum policy based on our values
    policy = calculate_greedy_policy(grid, V)

    # our goal here is to verify that we get the same answer as with policy iteration
    print("\nvalues:")
    print_values(V, grid)
    print("\npolicy:")
    print_policy(policy, grid)

In [8]:
grid = standard_grid(obey_prob=1.0, step_cost=None)
print_game(grid)

rewards:
---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|

values:
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|

policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |


#### obey_prob = 1.0 | step_down = None
Mario is not drunk it will follow what it said, it is most simple one

In [9]:
grid = standard_grid(obey_prob=0.8, step_cost=None)
print_game(grid)

rewards:
---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|

values:
---------------------------
 0.72| 0.83| 0.94| 0.00|
---------------------------
 0.63| 0.00| 0.64| 0.00|
---------------------------
 0.54| 0.48| 0.53| 0.31|

policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  L  |  U  |  L  |


#### obey_prob = 0.8 | step_down = None
Mario is little drunk, so it will try to avoid right path, so that chances of falling in pit is less

In [10]:
grid = standard_grid(obey_prob=0.5, step_cost=None)
print_game(grid)

rewards:
---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|

values:
---------------------------
 0.48| 0.63| 0.77| 0.00|
---------------------------
 0.39| 0.00| 0.44| 0.00|
---------------------------
 0.30| 0.24| 0.30| 0.20|

policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  L  |     |
---------------------------
  U  |  L  |  U  |  D  |


#### obey_prob = 0.5 | step_down = None
Mario is lot drunk, it will try to follow left path as it is safe. and on (1,2) it will start bumping into wall just to avoid pit. We can try to predict best move there
* up = (0.5x0 + 0.5x0.9x0.77) + (0.25x0+ 0.25x0.9x.44) + (0.25x-1 + 0.25x0.9x0 ) = 0.3465+0.099-1 = -0.5545
* left = (0.5x0 + 0.5x0.9x0.44 )+ (0.25x0+ 0.25x0.9x0.77) + (0.25x0+ 0.25x0.9x0.30) = 0.198+0.17325+0.0675 = 0.43875
* So, Left seem to be more rewarding

In [11]:
grid = standard_grid(obey_prob=0.5, step_cost=-0.1)
print_game(grid)

rewards:
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|

values:
---------------------------
-0.03| 0.27| 0.55| 0.00|
---------------------------
-0.21| 0.00|-0.10| 0.00|
---------------------------
-0.39|-0.50|-0.38|-0.57|

policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  D  |


#### obey_prob = 0.5 | step_down = -0.1
Mario is still lot drunk but it will have to reach goal as soon as possible, it will try to follow left path as it is safe. This time mario will take a risk on (1,2). We can try to predict best move there
* up = (0.5x-0.1 + 0.5x0.9x0.55) + (0.25x-.10+ 0.25x0.9x-.10) + (0.25x-1 + 0.25x0.9x0 ) = 0.2965-0.0475-0.25 = -0.001
* left = (0.5x-0.1 + 0.5x0.9x-0.1 )+ (0.25x-0.1+ 0.25x0.9x0.55) + (0.25x-0.1+ 0.25x0.9x-0.38) = -0.095+0.09875-0.1105 = -0.10675
* So, here up seems to be more rewarding
* weird thing happened at (2,3), mario seem to prefer in stay in same state rather than moving, as staying is more rewarding.

In [12]:
grid = standard_grid(obey_prob=0.5, step_cost=-0.5)
print_game(grid)

rewards:
---------------------------
-0.50|-0.50|-0.50| 1.00|
---------------------------
-0.50| 0.00|-0.50|-1.00|
---------------------------
-0.50|-0.50|-0.50|-0.50|

values:
---------------------------
-1.82|-0.82| 0.11| 0.00|
---------------------------
-2.40| 0.00|-0.74| 0.00|
---------------------------
-2.66|-2.28|-1.68|-1.45|

policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  R  |  R  |  U  |  U  |


#### obey_prob = 0.5 | step_down = -0.5
Mario is still lot drunk but it will have to reach goal as soon as possible such that again a weired thing happened at (2,3), mario decided to commit sucide. Lets see why
* up = ( 0.5x-1 + 0.5x0.9x0 )+( 0.25x-0.5 + 0.25x0.9x-1.68 )+(0.25x-0.5+0.25x0.9x-1.45) = -0.5-0.503-0.45125 = -1.45425
* down = (0.5x-0.5+0.5x0.9x-1.45)+(0.25x-0.5+0.25x0.9x-1.45)+(0.25x-0.5+0.25x0.9x-1.68) = -0.9025-0.45125-0.503 =-1.85675
* As it is clear, sucide is best option for mario

In [13]:
grid = standard_grid(obey_prob=1, step_cost=-2)
print_game(grid)

rewards:
---------------------------
-2.00|-2.00|-2.00| 1.00|
---------------------------
-2.00| 0.00|-2.00|-1.00|
---------------------------
-2.00|-2.00|-2.00|-2.00|

values:
---------------------------
-2.99|-1.10| 1.00| 0.00|
---------------------------
-4.69| 0.00|-1.00| 0.00|
---------------------------
-6.15|-4.61|-2.90|-1.00|

policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  R  |  R  |  U  |  U  |


#### obey_prob = 0.5 | step_down = -2
Mario is still lot drunk but it will have to reach goal as soon as possible and such that moving one extra tile is having more penalty which cant be compensated by any future reward. So here also mario prefers to die in some of the situations. Poor Mario!!!....